# Ackerberg sampling and onwards

In this notebook, we simulate out the games, create importance weights, and do a lot of other juicy stuff to get ready for estimation. There is a lot of looping and programming here, so this could get long, but let's try to keep things as transparent as possible. 

We should describe the essence of Ackerberg sampling here as well, so that we can produce a cogent explanation whenever someone asks. 

The first steps are basically the same as usual:

In [1]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

In [2]:
%%stata -s nh1
cd C:\Users\mjbaker\Documents\GitHub\NewsHour
clear all
use AveragedDataDyno.dta
set more off
set seed 566971


C:\Users\mjbaker\Documents\GitHub\NewsHour


The first thing we will do is generate (if it is not already there) a marker for who is in the game and who is not - **note**: this is perhaps where branchings of our work will begin in the future. So:

In [3]:
%%stata -s nh1
capture gen game= (lnews | otherl) & lnpps!=.
sort market stationid mt timeslot

Note that the above code reflects the idea that only those stations broadcasting local news or other local programming (entertainment programming) in the actual paper are "making decisions" at any given time. Also, we need to have a price variable for them as well. 

The usual next step is to pull everything we need into `Mata`, while also generating any additional variables that we might need. 

In [4]:
%%stata -s nh1 --mata
    st_view(lnews=.,.,"lnews")
    st_view(nnews=.,.,"nnews")
    st_view(otherl=.,.,"otherl")
    st_view(otherc=.,.,"otherc")
    st_view(si=.,.,"si")
    st_view(mt=.,.,"mt")
    st_view(id=.,.,"stationid")
    st_view(M=.,.,"market")
    st_view(time=.,.,"timeslot")
    st_view(game=.,.,"game")
    st_view(lnpps=.,.,"lnpps")
    st_view(pop=.,.,"ACS_HH") 
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    st_view(dln=.,.,"dln")

    lnviewn=ln(pop:*si)
    m=panelsetup(M,1)

Mata output:

:     st_view(lnews=.,.,"lnews")

:     st_view(nnews=.,.,"nnews")

:     st_view(otherl=.,.,"otherl")

:     st_view(otherc=.,.,"otherc")

:     st_view(si=.,.,"si")

:     st_view(mt=.,.,"mt")

:     st_view(id=.,.,"stationid")

:     st_view(M=.,.,"market")

:     st_view(time=.,.,"timeslot")

:     st_view(game=.,.,"game")

:     st_view(lnpps=.,.,"lnpps")

:     st_view(pop=.,.,"ACS_HH") 

:     st_view(l_ACS_HH=.,.,"l_ACS_HH")

:     st_view(lnewsn=.,.,"lnewsn")

:     st_view(otherln=.,.,"otherln")

:     st_view(nnewsn=.,.,"nnewsn")

:     st_view(othercn=.,.,"othercn")

:     st_view(dln=.,.,"dln")

:     m=panelsetup(M,1)


Next we will get the starting values organized and set them up, as they were estimated in the previous two notebooks. 

Pull in the starting values and get them all set up. Note this requires a little bit of rearranging and some transformation to suit the way we entered things into the likelihood. We also set the number of draws at 20 - while this might seem low, it is hard to do more because the problem is so giant!

In [5]:
%%stata -s nh1 --mata
    mata matuse DynoStarts23

    bo[,1::4]=bo[,1::4]:^2
    bo=bo[rows(bo),]

    betaDynoStart=bo[(5..23,27)]

    sdstav=exp(bo[24])
    sdmarv=exp(bo[25])
    sdmodv=exp(bo[26])
    
    draws=20

Mata output:

:     mata matuse DynoStarts23
(loading arate[1,27], bo[1,27], drawsbo[50,27], vals[50,1])

:     bo=bo[rows(bo),]

:     sdmarv=exp(bo[25])

:     sdmodv=exp(bo[26])

:     
:     draws=20


In a previous workbook, we did a big reshape and some associated calculations to get the data in mata form, and in reshaped form. That is, so we have stations as observations, with each row representing a share, price, decision, etc. of dimension 6 - the number of time periods. 

This is commonly referred to as wide format in Stata, which, for some reason, I decided to mark by suffixing things with the `Long`. To keep things short, I gathered all this code (it is redundant) into a script called `MataReshape.do`. The resulting variables are defined exactly as they were in a previous workbook. 

In [6]:
%%stata -s nh1 -np
do MataFunctions\MataReshape.do

## Viewership error terms

That having been done, we now have to recover actual viewership errors, given our model parameters. More correctly, what we really need is the utility index for viewership. This is pretty easy to calculate given our model estimates. If we have (ignoring subscripts, etc. just to give the basic idea):

$$
\ln(s_i)-\ln(s_0) = \mu \ln(s_{ig})+U_i
$$

where $s_{ig}$ is the share of $i$ in $i$'s programming group. 

We can get $U_i=X\beta_v+\epsilon$ by calculating:

$$
U_i = \ln(s_i)-\ln(s_0)-\mu \ln (s_{ig})
$$

In St(M)ata, we calculate the within group share (even though we have it - all these redundancies are good for checking things are working), and then calculate this linear viewership term. We also include a calculation that includes everything in the $X\beta$ vector, including error terms. 

In [7]:
%%stata -s nh1 --mata
    swgLong=siLong:/(slnewsLong:*lnewsLong:+
        snnewsLong:*nnewsLong:+
        sotherlLong:*otherlLong:+
        sothercLong:*othercLong)
    
    lnswgLong=ln(swgLong)   

    dln=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong))    
    
    XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
        bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong
                
    xBpUE=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong)):-
        bo[1]:*lnswgLong:*lnewsLong:-
        bo[2]:*lnswgLong:*otherlLong-
        bo[3]:*lnswgLong:*nnewsLong:-
        bo[4]:*lnswgLong:*othercLong
                
    xBDyno=J(rows(lnewsLong),cols(lnewsLong),0)
    for (i=1;i<=cols(lnewsLong);i++) {
        xDyno=lnewsLong[,i],otherlLong[,i],nnewsLong[,i],
            lnewslnews[,i],lnewsnnews[,i],nnewslnews[,i],nnewsnnews[,i],
            siLongLag[,i],siXlnln[,i],siXlnnn[,i],siXnnln[,i],siXnnnn[,i],
            lnewsRT[,i],nnewsRT[,i],l_ACS_HHLong[,i],lnewsnLong[,i],otherlnLong[,i],
            nnewsnLong[,i],othercnLong[,i],J(rows(lnewsLong),1,1)
        xBDyno[,i]=xDyno*betaDynoStart'
    }              

Mata output:

:     swgLong=siLong:/(slnewsLong:*lnewsLong:+
>         snnewsLong:*nnewsLong:+
>         sotherlLong:*otherlLong:+
>         sothercLong:*othercLong)

:     
:     lnswgLong=ln(swgLong)   

:     
:     XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
>         bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong

:                 
:     xBpUE=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong)):-
>         bo[1]:*lnswgLong:*lnewsLong:-
>         bo[2]:*lnswgLong:*otherlLong-
>         bo[3]:*lnswgLong:*nnewsLong:-
>         bo[4]:*lnswgLong:*othercLong

:                 
:     xBDyno=J(rows(lnewsLong),cols(lnewsLong),0)

:     for (i=1;i<=cols(lnewsLong);i++) {
>         xDyno=lnewsLong[,i],otherlLong[,i],nnewsLong[,i],
>             lnewslnews[,i],lnewsnnews[,i],nnewslnews[,i],nnewsnnews[,i],
>             siLongLag[,i],siXlnln[,i],siXlnnn[,i],siXnnln[,i],siXnnnn[,i],
>             lnewsRT[,i],nnewsRT[,i],l_ACS_HHLong[,i],lnew

In [8]:
%%stata -s nh1 --mata
mata: XBVngLong[1::10,]
mata: hasmissing(XBVngLong)

Mata output:

                   1              2              3              4              5              6
     +-------------------------------------------------------------------------------------------+
   1 |   -8.98617853   -8.393099462   -8.732006689   -8.434466056   -6.958609063   -6.845815918  |
   2 |  -8.151506762   -8.357819538   -8.689404731   -8.615718811   -7.967011578    -7.96631088  |
   3 |  -8.236010271   -8.282748201   -8.344985832   -8.461517439   -8.280821686   -8.178516242  |
   4 |  -9.450739205     -9.1754815   -9.431531418   -8.853593218   -9.088094201   -9.139364433  |
   5 |  -9.178572108   -9.092025042   -8.417539311   -8.300364258   -8.366669511   -8.269470259  |
   6 |  -8.637249246   -8.510646998   -9.197867249   -8.994572027   -7.609329363    -7.51342862  |
   7 |  -8.743443553   -8.628063965    -8.73036568   -8.358573904   -8.014290152   -7.948345372  |
   8 |  -8.186018297   -7.863817764     -7.5394844   -7.446144762   -7.031097822   -6.766959458  |

These seem about right. so, now what we can do is start drawing random effects and all that. To do this, we can write a little function called `paneldraw()` which draws common values across a user-specified panel identifier. I've written this function so that it works with the id only, and doesn't require the data to be sorted. 

Since we will need this function again, we will do our adopted trick of writing the results to a file and then running the file. 

In [9]:
CWD = os.getcwd()
os.chdir(CWD + '\\MataFunctions')

In [10]:
%%writefile paneldraw.do
mata: 
    real matrix paneldraw(real matrix id, real scalar draws)
    {
        rowsToDraw=uniqrows(id)

        U=J(rows(id),draws,.)
        for (i=1;i<=rows(rowsToDraw);i++) {
            obs=mm_which(rowsToDraw[i]:==id)
            D=runiform(1,draws)
            U[obs,]=J(rows(obs),1,D)
        }
        return(U)
    }
end
end

Overwriting paneldraw.do


In [11]:
%%stata -s nh1 
do MataFunctions\paneldraw.do


:     real matrix paneldraw(real matrix id, real scalar draws)
>     {
>         rowsToDraw=uniqrows(id)
> 
>         U=J(rows(id),draws,.)
>         for (i=1;i<=rows(rowsToDraw);i++) {
>             obs=mm_which(rowsToDraw[i]:==id)
>             D=runiform(1,draws)
>             U[obs,]=J(rows(obs),1,D)
>         }
>         return(U)
>     }

-------------------------------------------------------------------------------------------------------------------------------------------------

end of do-file


**Note that we are now using stata in one directory, but Python is working in the directory `MataFunctions` **

Note that this just provides uniform random variates with common values across panel variables. We just use our estimated parameters and the usual inverse-normal transformation to get the actual values. We then reshape these into a useable form and call the results `UvsLong` and `UvmtLong` - the former being the station random effect, the latter being the market random effect. 

As the last part of the code block, we draw a purely idiosynchratic error term. 

Note that the stata package `moremata` is required for this step.

In [12]:
%%stata -s nh1 --mata
    iev1=paneldraw(id,draws)
    iev2=paneldraw(mt,draws)
    iev1=sdstav*invnormal(iev1)
    iev2=sdmarv*invnormal(iev2)

    UvsLong=J(rows(lnewsLong),0,0)
    UvmtLong=J(rows(lnewsLong),0,0)
    
    for (i=1;i<=draws;i++) {
        UvsLong=UvsLong,colshape(iev1[,i],timeslots)
        UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
    }

Mata output:

:     iev1=paneldraw(id,draws)

:     iev2=paneldraw(mt,draws)

:     iev1=sdstav*invnormal(iev1)

:     iev2=sdmarv*invnormal(iev2)

:     UvmtLong=J(rows(lnewsLong),0,0)

:     
:     for (i=1;i<=draws;i++) {
>         UvsLong=UvsLong,colshape(iev1[,i],timeslots)
>         UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
>     }


To keep stuff outside the loop outside the cell:

In [13]:
%%stata -s nh1 --mata
    UvmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodv

Mata output:

:     UvmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodv


The next thing to be done is to calculate what amount to "observed" error terms for the viewership model (as opposed to simulated error terms from things we haven't observed). These are basically whatever is left over after fitting the model.
I will calculate all this out, and then expand things into a sample that conforms with the draws. 

However, note that the end result **DOES** vary, because we do not observe the division of the error term into market-time, station, and idiosyncratic effects. The last line subtracts out our sampled two level random effects.

In [14]:
%%stata -s nh1 --mata
    UvmodObsLong=xBpUE:-xBDyno
    UvmodObsLong=J(1,draws,1)#UvmodObsLong:-UvsLong:-UvmtLong

Mata output:

:     UvmodObsLong=xBpUE:-xBDyno

:     UvmodObsLong=J(1,draws,1)#UvmodObsLong:-UvsLong:-UvmtLong


## Price error terms

So, we now have a full set of (observed) error terms for the viewership model. Once again, these are "observed" in the sense that they derive directly from differences between dependent and independent variables. 

We will now turn to dealing with the price model. The first step is loading up our estimated price model and its estimated coefficients, and then transforming and collecting the estimated variance terms. Here we go:

In [15]:
%%stata -s nh1 --mata
    mata matuse betaPDynoStarts, replace
    
    sdstap=exp(bpo[7])
    sdmarp=exp(bpo[8])
    sdmodp=exp(bpo[9])    

Mata output:

:     mata matuse betaPDynoStarts, replace
(loading bpo[1,10], drawsbpo[200,10])

:     
:     sdstap=exp(bpo[7])

:     sdmarp=exp(bpo[8])

:     sdmodp=exp(bpo[9])    


Now, we need to get random effects in order for the pricing equation:

In [16]:
%%stata -s nh1 --mata
    iep1=paneldraw(id,draws)
    iep2=paneldraw(mt,draws)
    iep1=sdstap*invnormal(iep1)
    iep2=sdmarp*invnormal(iep2)	

    UpsLong=J(rows(lnewsLong),0,0)
    UpmtLong=J(rows(lnewsLong),0,0)
    
    for (i=1;i<=draws;i++) {
        UpsLong=UpsLong,colshape(iep1[,i],timeslots)
        UpmtLong=UpmtLong,colshape(iep2[,i],timeslots)
    }
    
    UpmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodp

Mata output:

:     iep1=paneldraw(id,draws)

:     iep2=paneldraw(mt,draws)

:     iep1=sdstap*invnormal(iep1)

:     iep2=sdmarp*invnormal(iep2)

:     UpmtLong=J(rows(lnewsLong),0,0)

:     
:     for (i=1;i<=draws;i++) {
>         UpsLong=UpsLong,colshape(iep1[,i],timeslots)
>         UpmtLong=UpmtLong,colshape(iep2[,i],timeslots)
>     }

:     
:     UpmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodp


Save draws of random effects for later use...

In [17]:
%%stata -s nh1 --mata
mata matsave REffects UpsLong UpmtLong UvsLong UvmtLong, replace

Mata output:

(saving UpmtLong[16622,120], UpsLong[16622,120], UvmtLong[16622,120], UvsLong[16622,120])
file REffects.mmat saved


## **Point of deviation!**

At this point, we are deviating from the model as it currently appears in the paper. Given the comments of discussants at various presentations, we felt that it would be useful to take heed to one critique, given that we do not have data on the relative costs of broadcasting local news or entertainment, and that people feel that our results are "too large" in terms of welfare effects.

### First Modification

One reason this might be the case is that we have been assuming throughout that **_any_** local station could switch to local news. This might be excessive, so we modifying things to allow only stations that have broadcasted local news at some point during the day to broadcast local news (i.e., these are the stations that have a demonstrated ability to broadcast news). 

Here is our original code that does this:

In [18]:
%%stata -s nh1 --mata
    gameMarker = rowsum(gameLong):>0
    sum(gameMarker)

Mata output:

:     gameMarker = rowsum(gameLong):>0

:     sum(gameMarker)
  623


We see that 623 stations, across all markets, are decision-makers. We adjust to zero firms that never broadcast local news (they are now "out of the game"). We also create a conformable version of `decCounter`, which counts the number of periods in which a station can make decisions. We also adjust our `gameLong` marker. 

How many of these stations never broadcast local news? 

In [19]:
%%stata -s nh1 --mata
    lnewsSome  = rowsum(lnewsLong):>0
    gameMarker = gameMarker:*lnewsSome
    gameLong   = gameLong:*gameMarker
    decCounter = rowsum(gameLong)
    
    sum(gameMarker)

Mata output:

:     lnewsSome  = rowsum(lnewsLong):>0

:     gameMarker = gameMarker:*lnewsSome

:     gameLong   = gameLong:*gameMarker

:     decCounter = rowsum(gameLong)

:     
:     sum(gameMarker)
  400


So, we see that the set of decision-making stations per our model is reduced by about 1/3. Let's proceed along these lines now. 

The basic idea now is to create for each station a set of arrays that store error terms and draws, potential strategy sets (remembering that national news is fixed), and simulated shares from unilateral deviations. We will carry these around in three associative arrays (which can be saved as-is for use in the sequel). 

## Potential strategies

We first go through the process of creating an array of potential strategies that each station might adopt. These are typically of dimension 5 or 6 which coincides with the number of periods the station can make decisions in (national news if fixed, recall). That is, each station has 5 consecutive programming decisions to make, so can choose from about $2^5=32$ or $2^6=64$ potential strategies. 

**Note: We have not yet filled out the `uvInfo` but are keeping it in here as a reminder. This is where we will store information relevant to the AKW importance sampler. **

In [20]:
%%stata -s nh1 --mata
    Bcs      =asarray_create("real", 2)


Mata output:

:     Bcs      =asarray_create("real", 2)


We also need a little function that can delineate all the potential strategies as they consist of ones and zeros. The function `stratmat()` does this. To give credit where credit is due, I believe I borrowed the gist of the code from an old Stata Journal piece by Nick Cox - I can't remember which edition, but I would dig it up if someone asked for it!

I also give an example with four possible strategies just to be sure it's working. 

In [21]:
%%writefile stratmat.do
mata:
real matrix stratmat(real scalar x)
{
    real matrix M,MM,V
    real colvector p
    real scalar info,j

    V=J(x,x,1)
    V=J(rows(V),1,0),lowertriangle(V)
    MM=J(rows(V),1,.)
    for (j=1;j<=cols(V);j++) {
        info=cvpermutesetup(V[.,j])
        M=J(rows(V),1,.)
        while ((p=cvpermute(info)) !=J(0,1,.)) {
            M=M,p
        }
    M=M[.,2::cols(M)]
    MM=MM,M
    }
    MM=MM[.,2::cols(MM)]
    
    return(MM)
} 
end
end

Overwriting stratmat.do


In [22]:
%%stata -s nh1 
do MataFunctions\stratmat.do
mata: stratmat(4)


> {
>     real matrix M,MM,V
>     real colvector p
>     real scalar info,j
> 
>     V=J(x,x,1)
>     V=J(rows(V),1,0),lowertriangle(V)
>     MM=J(rows(V),1,.)
>     for (j=1;j<=cols(V);j++) {
>         info=cvpermutesetup(V[.,j])
>         M=J(rows(V),1,.)
>         while ((p=cvpermute(info)) !=J(0,1,.)) {
>             M=M,p
>         }
>     M=M[.,2::cols(M)]
>     MM=MM,M
>     }
>     MM=MM[.,2::cols(MM)]
>     
>     return(MM)
> } 

-------------------------------------------------------------------------------------------------------------------------------------------------

end of do-file
        1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16
    +---------------------------------------------------------------------------------+
  1 |   0    1    0    1    1    1    0    0    0    1    1    1    0    0    0    1  |
  2 |   0    1    1    0    1    1    0    1    1    0    0    1    0    0    1    0  |
  3 |   0    1    1    1    0    1    1    

The above is just a brief illustration, but what we will do is use the ones to denote local news, the zeros to denote other programming. 

Having created the associative arrays, we now forge onwards and pick out our competitors' action vectors ids, and total number of periods in which decisions can be made. Note that if only five decisions are made, we know that a national news broadcast occurs at some point in the broadcast evening. 

Note that since each `gameIds` is just a number marking the station, I just pick out the first of these numbers for storage in the associative array. 

So, `lnewsBcs`, is a set of dummies indicating whether or not local news is broadcast, as is the case for `nnewsBcs` and `otherlBcs`. `totalDecs` adds up how many total decisions are being made. 

In [23]:
%%stata -s nh1 --mata
    gameIds=select(statIdLong,gameMarker)
    lnewsBcs=select(lnewsLong,gameMarker)
    nnewsBcs=select(nnewsLong,gameMarker)
    otherlBcs=select(otherlLong,gameMarker)
    totalDecs=select(decCounter,gameMarker)
    
    for (i=1;i<=rows(gameIds);i++) {
        if (totalDecs[i]==6) {
            strats=stratmat(6)'
            asarray(Bcs,(gameIds[i,1],1),strats)
            asarray(Bcs,(gameIds[i],2),1:-strats)
            asarray(Bcs,(gameIds[i],3),J(rows(strats),cols(strats),0))
        }
        if (totalDecs[i]==5) {
            nnewsPos=mm_which(nnewsBcs[i,1::6])
            stratsFree=stratmat(5)'
            if (nnewsPos==1) lnewsp=J(rows(stratsFree),1,0),stratsFree
            else if (nnewsPos==6) lnewsp=stratsFree,J(rows(stratsFree),1,0)
            else lnewsp=stratsFree[,1::nnewsPos-1],J(rows(stratsFree),1,0),stratsFree[,nnewsPos::5]
            otherlp=1:-lnewsp
            otherlp[,nnewsPos]=J(rows(otherlp),1,0)
            nnewsp=J(rows(lnewsp),6,0)
            nnewsp[,nnewsPos]=J(rows(lnewsp),1,1)
            asarray(Bcs,(gameIds[i],1),lnewsp)
            asarray(Bcs,(gameIds[i],2),otherlp)
            asarray(Bcs,(gameIds[i],3),nnewsp)
        }
    }

Mata output:

:     gameIds=select(statIdLong,gameMarker)

:     lnewsBcs=select(lnewsLong,gameMarker)

:     nnewsBcs=select(nnewsLong,gameMarker)

:     otherlBcs=select(otherlLong,gameMarker)

:     totalDecs=select(decCounter,gameMarker)

:     
:     for (i=1;i<=rows(gameIds);i++) {
>         if (totalDecs[i]==6) {
>             strats=stratmat(6)'
>             asarray(Bcs,(gameIds[i,1],1),strats)
>             asarray(Bcs,(gameIds[i],2),1:-strats)
>             asarray(Bcs,(gameIds[i],3),J(rows(strats),cols(strats),0))
>         }
>         if (totalDecs[i]==5) {
>             nnewsPos=mm_which(nnewsBcs[i,1::6])
>             stratsFree=stratmat(5)'
>             if (nnewsPos==1) lnewsp=J(rows(stratsFree),1,0),stratsFree
>             else if (nnewsPos==6) lnewsp=stratsFree,J(rows(stratsFree),1,0)
>             else lnewsp=stratsFree[,1::nnewsPos-1],J(rows(stratsFree),1,0),stratsFree[,nnewsPos::5]
>             otherlp=1:-lnewsp
>             otherlp[,nnewsPos]=J(rows(otherlp),1

Save results for later use...

In [24]:
%%stata -s nh1 --mata
mata matsave BCS Bcs, replace

Mata output:

(saving Bcs)
file BCS.mmat saved


First, let's set up the variables that we need to do this. We have the following variables that need to be set up before the loop begins, and we also need to load up our functions for calculating shares, which we called `sharemakers.do`. Recall, this group of `Mata` routines created methods for calculating shares given a set of linear indices for the firms. 

Since we wrote that to a file, we can really just do this as follows...

In [25]:
%%stata -s nh1 --mata
    mLong   = panelsetup(marketIdLong,1)
    counter = 1

Mata output:

:     mLong   = panelsetup(marketIdLong,1)

:     counter = 1


In [26]:
%%stata -s nh1
do MataFunctions\sharemakers.do


:     real matrix eshares_up(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
>     {
>         sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
>         lu=exp(Uv:/(1:-sigg))
>         lu_l=colsum(lnews:*lu)
>         lu_n=colsum(nnews:*lu)
>         lu_o=colsum(otherl:*lu)
>         lu_c=colsum(otherc:*lu)
>         swg=lu:/(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc)
>         share=swg:*(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc):^(1:-sigg):/
>                   (1:+lu_l:^(1:-sigma_l):+lu_n:^(1:-sigma_n):+lu_o:^(1:-sigma_o):+lu_c:^(1:-sigma_c))
>         return(share)
>     }

:     
:     real matrix esharesStable(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
>     {
>         sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
>         lnLu=Uv:/(1:-sigg)
>         if (colsum(lnews)>0) {
>             lnU=select(lnLu,lnews)
>             max=max(lnU)
>             lu_l=exp(max):*colsum(exp(

# Shares from unilateral deviation, given draws 

Now, the rubber is really going to hit the road. In the next block of code, we have a series of nested loops that does this. Let's make a schematic of levels for this so we can see what is going on. 

* loop over draws
   * loop over markets
       - calculate the number of players/stations and only "do something" if there are actually players in the market
       - if we have players/stations:
           * get their data and their position in the data (broadcast decisions, error terms, X variables...
           * loop over each player, and:
              - calculate out XB and hypothetical XB's for each possible broadcast sequence
              - get XB's for everyone else in the market
              - simulate shares for each potential unilateral deviation. 
              - plop strategies and coincident simulated unilater deviations into the array
              
              
   

The first thing we will do is write some stand-alone functions that allow us to quickly simulate what happens when someone deviates. The following program: called `loopShareGenerator()` takes in an index in the panel, an index in the array of strategies available the player, and a time, and calculates the resulting share. This is useful in starting off drawing. 

While the function looks long, each block of code is relatively straightforward:

1. The `external` information refers to stuff that is not passed into the function or created internally. 
2. Up until the loop starts, the next few blocks of code just initialize the first-period variables needed by the loop.
3. The loop starts up by filling in the `X` variables associated with the model (note that this will have to be changed if the model is changed).
4. Then, we loop over six time periods, which allows us to fill in lags. 
5. Finally, while all shares are calculated, this routine returns only the share pertaining to a particular station (which occurs at row `place` in the list of stations)

In [27]:
%%writefile loopShareGenerator.do
mata:
    real scalar loopShareGenerator(place, p, T) {

        external counter
        external lnewsLongp, otherlLongp, nnewsLongp, othercLongp
        external lnewsHat, otherlHat, nnewsHat
        external l_ACS_HHLongp
        external UvsLongp, UvmodObsLongp,UvmtLongp,UvmodLongp,errVdraws
        external betaDynoStart, bo

        phlnewsLongp = lnewsLongp
        phnnewsLongp = nnewsLongp
        photherlLongp = otherlLongp

        lnewsLongp[place,]=lnewsHat[p,]
        otherlLongp[place,]=otherlHat[p,]
        nnewsLongp[place,]=nnewsHat[p,]
        
        lnewsLongLagp=J(rows(lnewsLongp),1,0)
        nnewsLongLagp=J(rows(lnewsLongp),1,0)
        otherlLongLagp=J(rows(lnewsLongp),1,0)
        siLagp=J(rows(lnewsLongp),1,0)
        totlnewsp=J(rows(lnewsLongp),1,0)
        totnnewsp=J(rows(nnewsLongp),1,0)
    
        lnewsnLongp=lnewsLongp:*ln(1:+colsum(lnewsLongp))
        otherlnLongp=otherlLongp:*ln(1:+colsum(otherlLongp))
        nnewsnLongp=nnewsLongp:*ln(1:+colsum(nnewsLongp))
        othercnLongp=othercLongp:*ln(1:+colsum(othercLongp))

        for (t=1;t<=T;t++) {
            if (t!=1) siLagp=ln(siLagp)
            XV=lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                lnewsLongLagp:*lnewsLongp[,t],
                nnewsLongLagp:*lnewsLongp[,t],
                lnewsLongLagp:*nnewsLongp[,t],
                nnewsLongLagp:*nnewsLongp[,t],
                siLagp,
                siLagp:*lnewsLongLagp:*lnewsLongp[,t],
                siLagp:*nnewsLongLagp:*lnewsLongp[,t],
                siLagp:*lnewsLongLagp:*nnewsLongp[,t],
                siLagp:*nnewsLongLagp:*nnewsLongp[,t],
                lnewsLongp[,t]:*ln(1:+totlnewsp),nnewsLongp[,t]:*ln(1:+totnnewsp),
                l_ACS_HHLongp[,t],lnewsnLongp[,t],otherlnLongp[,t],
                nnewsnLongp[,t],othercnLongp[,t],J(rows(lnewsLongp),1,1)

            XBV=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                UvsLongp[,counter+t-1]:+UvmodObsLongp[,counter+t-1]
            if (t==T) {
                XBV[place,]=XV[place,]*betaDynoStart':+UvmtLongp[place,counter+t-1]:+
                UvsLongp[place,counter+t-1]:+UvmodLongp[place,counter+t-1]:+errVdraws[1,6]
            }

            sharesP=esharesStable(XBV,lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
            siLagp=sharesP

            totlnewsp=totlnewsp:+J(rows(lnewsLongp),1,colsum(lnewsLongp[,t]:*sharesP))
            totnnewsp=totnnewsp:+J(rows(nnewsLongp),1,colsum(nnewsLongp[,t]:*sharesP))
            lnewsLongLagp=lnewsLongp[,t]
            nnewsLongLagp=nnewsLongp[,t]
            otherlLongLagp=otherlLongp[,t]
        }
        
        lnewsLongp = phlnewsLongp
        otherlLongp = photherlLongp
        nnewsLongp = phnnewsLongp

        return(sharesP[place])
}
end
end


Overwriting loopShareGenerator.do


In [27]:
%%stata -s nh1
do MataFunctions\loopShareGenerator.do


:     real scalar loopShareGenerator(place, p, T) {
> 
>         external counter
>         external lnewsLongp, otherlLongp, nnewsLongp, othercLongp
>         external lnewsHat, otherlHat, nnewsHat
>         external l_ACS_HHLongp
>         external UvsLongp, UvmodObsLongp,UvmtLongp,UvmodLongp,errVdraws
>         external betaDynoStart, bo
> 
>         phlnewsLongp = lnewsLongp
>         phnnewsLongp = nnewsLongp
>         photherlLongp = otherlLongp
> 
>         lnewsLongp[place,]=lnewsHat[p,]
>         otherlLongp[place,]=otherlHat[p,]
>         nnewsLongp[place,]=nnewsHat[p,]
>         
>         lnewsLongLagp=J(rows(lnewsLongp),1,0)
>         nnewsLongLagp=J(rows(lnewsLongp),1,0)
>         otherlLongLagp=J(rows(lnewsLongp),1,0)
>         siLagp=J(rows(lnewsLongp),1,0)
>         totlnewsp=J(rows(lnewsLongp),1,0)
>         totnnewsp=J(rows(nnewsLongp),1,0)
>     
>         lnewsnLongp=lnewsLongp:*ln(1:+colsum(lnewsLongp))
>         otherlnLongp=otherlLongp:*ln(1:+colsum(otherlLongp

We next create a function that creates prices and shares for a certain range of forward-looking time slots. This is useful in the recursive drawing of our error terms. The function also takes flags (`cutFlagV` and `cutFlagP`) which allow one to get the price or share back when the idiosyncratic term is not included. This is useful in creating bounds and keeping track of means. 

In [29]:
%%writefile PriceShareGenerator.do
mata:
    void PriceShareGenerator(real scalar cutT, cutFlagV, cutFlagP, real scalar place, S, P, allS) {
    
        external counter, timeslots, popp
        external lnewsLongp, otherlLongp, nnewsLongp, othercLongp
        external lnewsHat, otherlHat, nnewsHat
        external l_ACS_HHLongp
        external UvsLongp, UvmodObsLongp, UvmtLongp, UvmodLongp, 
            errVdraws, errPdraws, errMarker, UpmodobsLongp, UpmtLongp, UpsLongp
        external betaDynoStart, bo, bpo

        phlnewsLongp = lnewsLongp
        phnnewsLongp = nnewsLongp
        photherlLongp = otherlLongp
        phUvmodObsLongp = UvmodObsLongp
        phUpmodobsLongp = UpmodobsLongp

        shares = J(rows(lnewsLongp),0,.)

        UpmodObz=UpmodobsLongp[place,counter::counter+timeslots-1]
        UvmodObz=UvmodObsLongp[place,counter::counter+timeslots-1]

        if (cutT>1) {
            p=mm_which((rowsum(errMarker[,1::cutT-1]):==cutT-1):*(errMarker[,cutT]:==0))
            usePErrs=J(rows(p),timeslots,0)
            useVErrs=J(rows(p),timeslots,0)
     
            usePErrs[,1::cutT]=J(rows(p),1,UpmodObz[1,1::cutT])
            usePErrs[,cutT::cols(usePErrs)]=
                errMarker[p,cutT::cols(errPdraws)]:*UpmodObz[1,cutT::cols(errPdraws)]:+
                (1:-errMarker[p,cutT::cols(errPdraws)]):*errPdraws[,cutT::cols(errPdraws)]
                
            useVErrs[,1::cutT]=J(rows(p),1,UvmodObz[1,1::cutT])
            useVErrs[,cutT::cols(useVErrs)]=
                errMarker[p,cutT::cols(errVdraws)]:*UvmodObz[1,cutT::cols(errVdraws)]:+
                (1:-errMarker[p,cutT::cols(errVdraws)]):*errVdraws[,cutT::cols(errVdraws)]
        }
        else {
            p=mm_which(errMarker[,cutT]:==0)
            if (rows(p)==0) p=1::rows(errMarker)

            usePErrs=J(rows(p),timeslots,0)
            useVErrs=J(rows(p),timeslots,0)
            usePErrs[,cutT::cols(usePErrs)]=
                errMarker[p,cutT::cols(errPdraws)]:*UpmodObz[1,cutT::cols(errPdraws)]:+
                (1:-errMarker[p,cutT::cols(errPdraws)]):*errPdraws[,cutT::cols(errPdraws)]
                
            useVErrs[,cutT::cols(useVErrs)]=
                errMarker[p,cutT::cols(errVdraws)]:*UvmodObz[1,cutT::cols(errVdraws)]:+
                (1:-errMarker[p,cutT::cols(errVdraws)]):*errVdraws[,cutT::cols(errVdraws)]
       }

        if (cutFlagP == 1) usePErrs[,cutT]=J(rows(p),1,0)
        if (cutFlagV == 1) useVErrs[,cutT]=J(rows(p),1,0)

        Shares = J(rows(p),timeslots,0)

        for (k=1;k<=rows(p);k++) {
            UvmodObsLongp[place,counter::counter+timeslots-1] = useVErrs[k,]
            lnewsLongp[place,]=lnewsHat[p[k],]
            otherlLongp[place,]=otherlHat[p[k],]
            nnewsLongp[place,]=nnewsHat[p[k],]
        
            lnewsLongLagp=J(rows(lnewsLongp),1,0)
            nnewsLongLagp=J(rows(lnewsLongp),1,0)

            siLagp=J(rows(lnewsLongp),1,0)
            totlnewsp=J(rows(lnewsLongp),1,0)
            totnnewsp=J(rows(nnewsLongp),1,0)
    
            lnewsnLongp=lnewsLongp:*ln(1:+colsum(lnewsLongp))
            otherlnLongp=otherlLongp:*ln(1:+colsum(otherlLongp))
            nnewsnLongp=nnewsLongp:*ln(1:+colsum(nnewsLongp))
            othercnLongp=othercLongp:*ln(1:+colsum(othercLongp))	
        
            shares = J(rows(lnewsLongp),0,.)

            for (t=1;t<=timeslots;t++) {
                if (t!=1) siLagp=ln(siLagp)
                XV=lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                    lnewsLongLagp:*lnewsLongp[,t],
                    nnewsLongLagp:*lnewsLongp[,t],
                    lnewsLongLagp:*nnewsLongp[,t],
                    nnewsLongLagp:*nnewsLongp[,t],
                    siLagp,
                    siLagp:*lnewsLongLagp:*lnewsLongp[,t],
                    siLagp:*nnewsLongLagp:*lnewsLongp[,t],
                    siLagp:*lnewsLongLagp:*nnewsLongp[,t],
                    siLagp:*nnewsLongLagp:*nnewsLongp[,t],
                    lnewsLongp[,t]:*ln(1:+totlnewsp),nnewsLongp[,t]:*ln(1:+totnnewsp),
                    l_ACS_HHLongp[,t],lnewsnLongp[,t],otherlnLongp[,t],
                    nnewsnLongp[,t],othercnLongp[,t],J(rows(lnewsLongp),1,1)

                XBV=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                    UvsLongp[,counter+t-1]:+UvmodObsLongp[,counter+t-1]

                sharesP=esharesStable(XBV,lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                    othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
                siLagp=sharesP
                shares = shares, sharesP

            totlnewsp=totlnewsp:+J(rows(lnewsLongp),1,colsum(lnewsLongp[,t]:*sharesP))
            totnnewsp=totnnewsp:+J(rows(nnewsLongp),1,colsum(nnewsLongp[,t]:*sharesP))
            lnewsLongLagp=lnewsLongp[,t]
            nnewsLongLagp=nnewsLongp[,t]
        }

            Shares[k,.] = shares[place,]

        }

        actPayMean=(lnewsHat[p,]*bpo[1]:+otherlHat[p,]*bpo[2]:+nnewsHat[p,]*bpo[3]):*ln(popp:*Shares):+
            lnewsHat[p,]:*bpo[4]:+otherlHat[p,]*bpo[5]:+bpo[6]:*l_ACS_HHLongp[place,]:+bpo[10]:+
            UpsLongp[place,counter::counter+timeslots-1]:+UpmtLongp[place,counter::counter+timeslots-1]

        lnewsLongp = phlnewsLongp
        otherlLongp = photherlLongp
        nnewsLongp = phnnewsLongp
        UvmodObsLongp = phUvmodObsLongp  
    UpmodobsLongp = phUpmodobsLongp 
        
        S = Shares
        
        P = actPayMean+usePErrs
        
        allS = shares
        
    }
end
end

Overwriting PriceShareGenerator.do


In [28]:
%%stata -s nh1 
do MataFunctions\PriceShareGenerator.do


:     void PriceShareGenerator(real scalar cutT, cutFlagV, cutFlagP, real scalar place, S, P, allS) {
>     
>         external counter, timeslots, popp
>         external lnewsLongp, otherlLongp, nnewsLongp, othercLongp
>         external lnewsHat, otherlHat, nnewsHat
>         external l_ACS_HHLongp
>         external UvsLongp, UvmodObsLongp, UvmtLongp, UvmodLongp, 
>             errVdraws, errPdraws, errMarker, UpmodobsLongp, UpmtLongp, UpsLongp
>         external betaDynoStart, bo, bpo
> 
>         phlnewsLongp = lnewsLongp
>         phnnewsLongp = nnewsLongp
>         photherlLongp = otherlLongp
>         phUvmodObsLongp = UvmodObsLongp
>         phUpmodobsLongp = UpmodobsLongp
> 
>         shares = J(rows(lnewsLongp),0,.)
> 
>         UpmodObz=UpmodobsLongp[place,counter::counter+timeslots-1]
>         UvmodObz=UvmodObsLongp[place,counter::counter+timeslots-1]
> 
>         if (cutT>1) {
>             p=mm_which((rowsum(errMarker[,1::cutT-1]):==cutT-1):*(errMarker[,cutT]:==0))
> 

At this point, waiting around for programs to run can be a little frustrating. So, we made a script that at least fills us in that things are moving along. It will log whenever an iteration is completed, and send me a text message when done. 

We first set up a small block that just gets our information organized and ready to be stored. 


In [29]:
%%stata -s nh1 --mata
    counter = 1
    stata("set seed 5150")
    priceErrs   = asarray_create("real", 2)
    priceBounds = asarray_create("real", 2)
    viewErrs    = asarray_create("real", 2)
    viewBounds  = asarray_create("real", 2)
    sharesBCS   = asarray_create("real", 3)

Mata output:

:     counter = 1

:     stata("set seed 5150")

:     priceErrs   = asarray_create("real", 2)

:     priceBounds = asarray_create("real", 2)

:     viewErrs    = asarray_create("real", 2)

:     viewBounds  = asarray_create("real", 2)

:     sharesBCS   = asarray_create("real", 3)


This is also a convenient time to set up information that we need for the Ackerberg-Keane-Wolpin importance sampling strategy. Here, we need to keep track of the so-called U-terms, and mean values. Let's carefully write down what we have in mind here. While it might be confusing to try and save too much information at once, let's at least save:

1) simulated shares - index 1

2) simulated group shares - index 2

3) simulated other shares - index 3

Create observed pricing errors from the pricing model:

In [30]:
%%stata -s nh1 --mata
    UpmodObs=J(rows(lnewsLong),0,0)
    for (i=1;i<=draws*timeslots;i=i+timeslots) {
        UpmodObs=UpmodObs,lnppsLong:-lnewsLong:*lnviewnLong:*bpo[1]:-otherlLong:*lnviewnLong:*bpo[2]:-
            nnewsLong:*lnviewnLong:*bpo[3]:-lnewsLong:*bpo[4]:-otherlLong:*bpo[5]:-l_ACS_HHLong:*bpo[6]:-bpo[10]:-
            UpsLong[,i::i+timeslots-1]:-UpmtLong[,i::i+timeslots-1]
    }

Mata output:

:     UpmodObs=J(rows(lnewsLong),0,0)

:     for (i=1;i<=draws*timeslots;i=i+timeslots) {
>         UpmodObs=UpmodObs,lnppsLong:-lnewsLong:*lnviewnLong:*bpo[1]:-otherlLong:*lnviewnLong:*bpo[2]:-
>             nnewsLong:*lnviewnLong:*bpo[3]:-lnewsLong:*bpo[4]:-otherlLong:*bpo[5]:-l_ACS_HHLong:*bpo[6]:-bpo[10]:-
>             UpsLong[,i::i+timeslots-1]:-UpmtLong[,i::i+timeslots-1]
>     }


The next loop draws error terms successively, working backwards first from the implied bound on the viewership term, next on the bound on price. So that we can be sure its working...where is the stata log file? 

In [31]:
%%stata -s nh1 --mata
    for (c=1;c<=draws;c++) {
        for (i=1;i<=rows(mLong);i++) {
            gameMarkerp = panelsubmatrix(gameMarker,i,mLong)
            playersp = colsum(gameMarkerp)
            if (playersp>0) { 
                statIdLongp=panelsubmatrix(statIdLong,i,mLong)
                lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
                nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
                otherlLongp=panelsubmatrix(otherlLong,i,mLong)
                othercLongp=panelsubmatrix(othercLong,i,mLong)
                lnppsLongp=panelsubmatrix(lnppsLong,i,mLong)
                marketIdLongp = panelsubmatrix(marketIdLong,i,mLong)
                
                nsToChange=select(statIdLongp,gameMarkerp)
                
                UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
                UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
                UvsLongp=panelsubmatrix(UvsLong,i,mLong)
                UpsLongp=panelsubmatrix(UpsLong,i,mLong)
                UpmtLongp=panelsubmatrix(UpmtLong,i,mLong)
                UpmodobsLongp=panelsubmatrix(UpmodObs,i,mLong)
            
                XBVngLongp=panelsubmatrix(XBVngLong,i,mLong)
                UvmodObsLongp=panelsubmatrix(UvmodObsLong,i,mLong)
                l_ACS_HHLongp=panelsubmatrix(l_ACS_HHLong,i,mLong)
                popp = round(exp(max(l_ACS_HHLongp)))
                
                gameList = J(0,2,.)
                
                for (k=1;k<=rows(nsToChange);k++) { 
                    c,i,k
                    pId = nsToChange[k]
                    gameList = gameList \ (pId,i)
                    place=mm_which(statIdLongp:==pId)
                    lnewsHat=asarray(Bcs,(pId,1))
                    otherlHat=asarray(Bcs,(pId,2))
                    nnewsHat=asarray(Bcs,(pId,3))
                
                    lnewsAct=lnewsLongp[place,]
                    otherlAct=otherlLongp[place,]
                    nnewsAct=nnewsLongp[place,]
                
                    XBVplaceHold=J(rows(lnewsHat),cols(lnewsHat),.)
                    XBVplaceHoldMean=J(rows(lnewsHat),cols(lnewsHat),.)
                    
                    errMarker=(lnewsAct:==lnewsHat):*(otherlAct:==otherlHat)
                
                    errPdraws=J(1,timeslots,0)
                    errVdraws=J(1,timeslots,0)
                    pBound=J(1,timeslots,.)
                    vBound=J(1,timeslots,.)
                    vBound[timeslots] = 6.5                  
        
                    do {
                    
                        errVdraws[1,timeslots]=sdmodv*rnormal(1,1,0,1) 

                        p=mm_which(((rowsum(errMarker[,1::5])):==5):*(errMarker[,timeslots]:==0))  /*the sole deviate strategy */
                    
                        shareHat = loopShareGenerator(place, p, timeslots)
                    
                        actPayMean=(lnewsHat[p,]*bpo[1]:+otherlHat[p,]*bpo[2]:+nnewsHat[p,]*bpo[3]):*ln(popp:*shareHat):+
                            lnewsHat[p,]:*bpo[4]:+otherlHat[p,]*bpo[5]:+bpo[6]:*l_ACS_HHLongp[place,]:+bpo[10]:+
                            UpsLongp[place,counter::counter+timeslots-1]:+UpmtLongp[place,counter::counter+timeslots-1]

                        pBound[timeslots] = lnppsLongp[place,timeslots]-actPayMean[timeslots]
                    
                        errPdraws[,timeslots]=exp(bpo[9])*invnormal(runiform(1,1)*normal(pBound[timeslots]/exp(bpo[9])))

                        problemflag=0
                        for (t=timeslots-1;t>=1;t--) {
                            vBound[t] = 10
                            pBound[t] = 10
                            errVdraws[t] = vBound[t]
                            PriceShareGenerator(t, 0, 1, place, S=., P=., allShares = .)
                            check = rowsum(exp(lnppsLongp[place,t::timeslots]))-max(rowsum(exp(P[,t::timeslots])))

                            if (check < 0) {
                                Up   = 10
                                Down = -10
                                Dist = (Up - Down) / 2
                                XX   = (Up + Down) / 2
                                its = 0
                                do {
                                    Dist = Dist / 2
                                    errVdraws[t]= XX
                                    PriceShareGenerator(t, 0, 1, place, S=., P=., allShares = .)
                                    check = rowsum(exp(lnppsLongp[place,t::timeslots]))-max(rowsum(exp(P[,t::timeslots])))
                                    if (check < 0) {
                                        XX = XX - Dist
                                    }
                                    else {
                                        XX = XX + Dist
                                    }
                                    its++
                                } while (abs(check)>.01 & its < 40)
                            vBound[t] = XX
                            if (check<0) vBound[t] = vBound[t]-1.1
                            }
                            errVdraws[t] = sdmodv*invnormal(normal(vBound[t]/sdmodv)*runiform(1,1))
                            vBound[t] = 10
                            pBound[t] = 10
                            errVdraws[t] = vBound[t]
                            PriceShareGenerator(t, 0, 1, place, S=., P=., allShares=.)
                            check = rowsum(exp(lnppsLongp[place,t::timeslots]))-max(rowsum(exp(P[,t::timeslots])))

                            if (check < 0) {
                                Up   = 10
                                Down = -10
                                Dist = (Up - Down) / 2
                                XX   = (Up + Down) / 2
                                its = 0
                                do {
                                    Dist = Dist / 2
                                    errVdraws[t]= XX
                                    PriceShareGenerator(t, 0, 1, place, S=., P=., allShares=.)
                                    check = rowsum(exp(lnppsLongp[place,t::timeslots]))-max(rowsum(exp(P[,t::timeslots])))
                                    if (check < 0) {
                                        XX = XX - Dist
                                    }
                                    else {
                                        XX = XX + Dist
                                    }
                                    its++
                                } while (abs(check)>.01 & its < 40)
                            vBound[t] = XX
                            if (check<0) vBound[t] = vBound[t]-1.1
                            }
                            errVdraws[t] = sdmodv*invnormal(normal(vBound[t]/sdmodv)*runiform(1,1))
                            errPdraws[t] = sdmodp*invnormal(normal(pBound[t]/sdmodp)*runiform(1,1))
                        }
                        checkerator = (nnewsAct:!=1):*(pBound:==.)
                        problemflag = rowsum(checkerator)>0
                        if (problemflag) {
                            printf("redrawing");displayflush()
                        }
                    } while (problemflag==1)
                asarray(priceErrs,(pId,c),errPdraws)
                asarray(priceBounds,(pId,c),pBound)
                asarray(viewErrs,(pId,c),errVdraws)
                asarray(viewBounds,(pId,c),vBound)
          
                s0 = 1:-colsum(allShares)
                sl = colsum(lnewsLongp:*allShares)
                sn = colsum(nnewsLongp:*allShares)
                so = colsum(otherlLongp:*allShares)
                sc = colsum(othercLongp:*allShares)
                sg=colsum(lnewsLongp[place,]:*sl:+nnewsLongp[place,]:*sn:+
                    otherlLongp[place,]:*so:+othercLongp[place,]:*sc)  
                asarray(sharesBCS,(pId,c,1),allShares[place,])
                asarray(sharesBCS,(pId,c,2),sg)
                asarray(sharesBCS,(pId,c,3),s0)
                }
            }
        }
        counter = counter + timeslots
    }

Mata output:

:     for (c=1;c<=draws;c++) {
>         for (i=1;i<=rows(mLong);i++) {
>             gameMarkerp = panelsubmatrix(gameMarker,i,mLong)
>             playersp = colsum(gameMarkerp)
>             if (playersp>0) { 
>                 statIdLongp=panelsubmatrix(statIdLong,i,mLong)
>                 lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
>                 nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
>                 otherlLongp=panelsubmatrix(otherlLong,i,mLong)
>                 othercLongp=panelsubmatrix(othercLong,i,mLong)
>                 lnppsLongp=panelsubmatrix(lnppsLong,i,mLong)
>                 marketIdLongp = panelsubmatrix(marketIdLong,i,mLong)
>                 
>                 nsToChange=select(statIdLongp,gameMarkerp)
>                 
>                 UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
>                 UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
>                 UvsLongp=panelsubmatrix(UvsLong,i,mLong)
>                 UpsLongp=panelsubm

In [32]:
%%stata -s nh1 
mata:    mata matsave PVDraws priceErrs priceBounds viewErrs viewBounds sharesBCS, replace



(saving priceBounds, priceErrs, sharesBCS, viewBounds, viewErrs)
file PVDraws.mmat saved


Now, we have to assembled all the sampling weights, distributions, etc. of all the data. We will pile all of this into a whole bunch of different places. So:

## Ackerberg-Keane-Wolpin importance sampling weights

### Part one - linear component

We now have a set of error terms drawn under the basic assumptions of the model. We now want to assemble these into importance weights. One of the more time-consuming aspects of all this is going to be checking for multiplicity, but we will get to that in due time. 
 
### How the sampler works

It is also good to just have a refresher in mind for how the AKW sampler works in practice. What we want to simulate is something like:
$$
\int H(e,\theta)f(e,\theta) de
$$
The problem is that while $f()# is some simple density, $H$ may be a difficult integral to deal with. So, to begin with, we introduce a change of variable:

$$
\int H(u(e,\theta_0))h(u,\theta) de du
$$

Where I have written this a bit differently and stressed the fact that the $u-$ terms come from some initial model, which I have interpreted to mean "a model that produces some initial parameter estimates, so that we can then easily draw a bunch of simulated $u$-terms. We can draw the u's using an importance sampler. That is:

$$
\int H(u(e,\theta_0)\frac{g(u,\theta_0)}{g(u,\theta_0)}h(u,\theta) du
$$

or 

$$
\sim \sum_s \frac{H(u_s)}{g(u_s)}h(u_s,\theta) du
$$

So what does this mean for our model?

### Our model:

Just so that we can be sure we have a firm idea what we are doing, let's take a little time to explain things. Our model, is, in essence, something like:

$$
\ln p' = \ln Ms' + X\beta_p + u^p_s + u^p_{mt} + e_{imt}
$$

and 

$$ 
\ln s' -\ln s_0' =\mu' \ln s'_{ig} +X\beta_v+u_s+u_{mt}+e_{imt} 
$$ 
 
We have to collect all these terms in an appropriate way - so we need simulated shares, group shares, and total shares, along with simulated error terms. We also have the bounds, which will figure into the weights.  

Some things are easy: the density of all the random effects are just normal. Everything else isn't that bad either - all other variables are conditionally truncated normal, where truncation points are implied through recursion. Fortunately, we saved all of these as we drew our model. 

The one difficulty comes from the nuances in determining the density of the share, the null share, and the group share in terms of the sampler. So let's take a look...

## Viewership importance sampling weights

The way to think about this is that we wish to write use a simulated $s'$ in the pricing equation, and to do so we need to create a simulated density for a simulated $s$. So, the question is: what is the density of $s$ and can it be written as a simple function of a bunch of linear indices? Well, yes - it and a few other terms. 

First, observe that $s$ can be written as (with a very reduced set of notation):

$$
s' = \frac{e^{\frac{U_i}{1-\mu}}}{\left(\sum_g e^{\frac{U_k}{1-\mu}}\right)^{\mu}}\frac{1}{\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}}
$$

$$
\ln s' = \frac{U_i}{1-\mu}-\mu \ln \left[ \sum_g e^{\frac{U_k}{1-\mu}} \right] - \ln \left[\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}\right]
$$
and observe that what we are really using is a simulated log share. So we want the density of the log of $s$. To form this, we need to know the Jacobian of the transformation. Note that it involves a linear index with mean $X\beta+u_s+u_{mt}+e_{mst}$. So conditional on other things, we have a normal random variate with mean $M_{smt}=x\beta+u_s+u_{mt}+e_{mst}$. 

Differentiating gives:

$$
\frac{d\ln s'}{d e} =\frac{1}{1-\mu} - \frac{\mu}{1-\mu}\frac{e^{\frac{U_i}{1-\mu}}}{\sum_g e^{\frac{U_k}{1-\mu}}}-\frac{e^{\frac{U_i}{1-\mu}}}{\left(\sum_g e^{\frac{U_k}{1-\mu}}\right)^{\mu}}\frac{1}{\sum_G(\sum e^{\frac{U_k}{1-\mu}})^{1-\mu}}
$$

So we can reduce this down to:

$$
\frac{d \ln s'}{de} = \frac{1-\mu s'_{ig}-(1-\mu)s'_i}{1-\mu}
$$

Now, since $f_e(e)de = f_s(\ln s')d\ln s'$, we get:

$$
f_s(\ln s') = f_e(e)\frac{de}{d\ln s'}
$$

or

$$
f_s(\ln s') = f_e(e)\frac{1}{\frac{d \ln s'}{de} } = f_e(e)\frac{1-\mu}{1-\mu s'_{ig}-(1-\mu)s'_i}
$$


So, our argument is as follows (or, the drawing sequence if one prefers):

### Drawing sequence:

1. Draw $u_{vmt}$ from $N(0,\sigma_{vm})$. Sampling weight is $N(u_{vmt}, \sigma_{vm0})$. 
2. Draw $u_{vs}$ from $N(0, \sigma_{vs})$. Sampling weight is $N(u_{vs}, \sigma_{vs0})$. 
3. Conditional on this, $\ln s$ has density $N(M_{smt},\sigma_{emt})\times \frac{1-\mu}{1-\mu s'_{ig}-(1-\mu)s'_i})$, where  $M_{smt}=x\beta+u_s+u_{mt}$, and has an upper truncation point.  
4. Draw $u_{ps}$ from $N(0, \sigma_{ps})$. Sampling weight is $N(u_{ps}, \sigma_{ps0}$. 
5. Draw $u_{pmt}$ from $N(0, \sigma_{pmt})$. Sampling weight is $N(u_{pmt}, \sigma_{u_pm0}$. 
6. Conditional on $u$'s and $ \ln s'$, $\ln p$  is $N(\ln Ms' + X\beta_p + u^p_s + u^p_{mt}, \sigma_{pmt})$ with an upper truncation point as well. This can also be used to form the sampling weight. 

To do this, we just have to assemble all of the weights appropriately. It helps to have a log-normal density function available:

In [33]:
%%writefile lnnd.do
mata
    real matrix lnnd(y,m,lnsd) return(-(y:-m):^2:/(2:*exp(lnsd):^2):-ln(2*pi())/2:-lnsd)   
end
end

Overwriting lnnd.do


In [34]:
%%stata -s nh1 
do MataFunctions\lnnd.do


:     real matrix lnnd(y,m,lnsd) return(-(y:-m):^2:/(2:*exp(lnsd):^2):-ln(2*pi())/2:-lnsd)   

-------------------------------------------------------------------------------------------------------------------------------------------------

end of do-file


Also, pull our other stuff back into existence if it isn't in memory:

In [35]:
%%stata -s nh1 --mata
mata matuse PVDraws, replace

Mata output:

(loading priceBounds, priceErrs, sharesBCS, viewBounds, viewErrs)


A secondary block of code computes the sampling weights for the market effects and all that jive. This is because there really isn't all that much to it. Note that what we do is split the density value over all the observations - this just makes computation a little easier as it won't involve finding the last observation of a group or anything like that. 

Here are some placeholders for our first set of `u` terms. The `uv` means they are associated with the viewership model, while the `g` suffix means that we are talking about the weight associated with the term. I think this confirms with Ackerberg's notation. 

## Probability Weights

The first thing to do is to just take care of the random effects and their sampling weights. This is easy enough because they aren't truncated or anything. But I do use something a little different here - I weight them to spread them over all observations, rather than just tacking them onto the last observation of a group. This is just easier in parallel computation. 


In [36]:
%%stata -s nh1 --mata
    
    upre1 = UpmtLong
    upre2 = UpsLong
    uvre1 = UvmtLong
    uvre2 = UvsLong
    
    uvre2Weights=J(rows(statIdLong),timeslots,.)
    for (i=1;i<=rows(mLong);i++) {
        gP=panelsubmatrix(gameLong,i,mLong)
        gPtotal=colsum(gP)
        weightsP=(gPtotal:*gP)
    uvre2Weights[mLong[i,1]::mLong[i,2],.]=weightsP
    }
    
    upre1g =lnnd(upre1,0,ln(sdstap)):*1:/rowsum(gameLong)     
    upre2g =lnnd(upre2,0,ln(sdmarp)):*1:/rowsum(uvre2Weights)
    uvre1g=lnnd(uvre1,0,ln(sdstav)):*1:/rowsum(gameLong)  
    uvre2g=lnnd(uvre2,0,ln(sdmarv)):*1:/rowsum(uvre2Weights)              

Mata output:

:     
:     upre1 = UpmtLong

:     upre2 = UpsLong

:     uvre1 = UvmtLong

:     uvre2 = UvsLong

:     
:     uvre2Weights=J(rows(statIdLong),timeslots,.)

:     for (i=1;i<=rows(mLong);i++) {
>         gP=panelsubmatrix(gameLong,i,mLong)
>         gPtotal=colsum(gP)
>         weightsP=(gPtotal:*gP)
>     uvre2Weights[mLong[i,1]::mLong[i,2],.]=weightsP
>     }

:     
:     upre1g =lnnd(upre1,0,ln(sdstap)):*1:/rowsum(gameLong)     

:     upre2g =lnnd(upre2,0,ln(sdmarp)):*1:/rowsum(uvre2Weights)

:     uvre1g=lnnd(uvre1,0,ln(sdstav)):*1:/rowsum(gameLong)  

:     uvre2g=lnnd(uvre2,0,ln(sdmarv)):*1:/rowsum(uvre2Weights)              


This gives us four terms: `upre1g`, `upre2g`, `uvre1g`, and `uvre2g` which are the sampling weights for our four random effects. Now, we also need a sampling weight for our simulated log-prices. This is a bit more involved. We have drawn values for all of the prices and their upper bounds. Let's unpack these and organize them in the same panel fashion as our previous information.  

The three entities that we need to do this are the associative arrays `priceErrs`, `priceBounds`, and `shares`. 

**Note:** One thing to note in the following loop is that I use the old "switching" trick because these are counterfactuals. That is, I essentially switch the values of `lnews` and the values of `otherl` in the linear prediction equations, because if the station actually broadcasted `lnews`, it means that the counterfactual involves broadcasting `otherl`, and vice versa.

We will begin by just making a few placeholders for our variables. Here they are:

In [37]:
%%stata -s nh1 --mata
    up  = J(rows(statIdLong), timeslots*draws, 0)
    upg = J(rows(statIdLong), timeslots*draws, 0)
    upb = J(rows(statIdLong), timeslots*draws, 0)
    usi = J(rows(statIdLong), timeslots*draws, 0)
    usg = J(rows(statIdLong), timeslots*draws, 0)

Mata output:

:     up  = J(rows(statIdLong), timeslots*draws, 0)

:     upg = J(rows(statIdLong), timeslots*draws, 0)

:     upb = J(rows(statIdLong), timeslots*draws, 0)

:     usi = J(rows(statIdLong), timeslots*draws, 0)

:     usg = J(rows(statIdLong), timeslots*draws, 0)


## Loop to fill in Hypothetical Prices 

In [38]:
%%stata -s nh1 --mata
looper = 1
    for (c=1; c<=draws; c++ ) {
        for (i=1; i<=rows(statIdLong); i++) {
            pId = statIdLong[i]
            if (gameMarker[i] == 1) {
                siHat   = asarray(sharesBCS,(pId,c,1))
                sgHat   = asarray(sharesBCS,(pId,c,2))
                pErr    = asarray(priceErrs,(pId,c))
                pB      = asarray(priceBounds,(pId,c))
            
                lnewsp  = lnewsLong[i, ]
                otherlp = otherlLong[i, ]
                nnewsp  = nnewsLong[i, ]
            
                pop    = popLong[i, ]
                logPop = l_ACS_HHLong[i,]
            
                upmt  = upre1[i, looper::looper + timeslots - 1]
                ups = upre2[i, looper::looper + timeslots - 1]
            
                meanPHat = otherlp*bpo[1]:*ln(siHat:*pop) :+ lnewsp*bpo[2]:*ln(siHat:*pop) :+ nnewsp*bpo[3]:*ln(siHat:*pop) :+
                    bpo[4]:*otherlp :+ bpo[5]:*lnewsp :+ bpo[6]:*logPop :+ ups :+ upmt :+ bpo[10] 
                pHat = meanPHat :+ pErr
                gup =  lnnd(pErr, 0, ln(sdmodp)) :- ln(normal(pB))
            
                up[i, looper::looper + timeslots - 1]  = pHat
                upg[i, looper::looper + timeslots - 1] = gup
                upb[i, looper::looper + timeslots - 1] = meanPHat :+ pB
                usi[i, looper::looper + timeslots - 1] = siHat
                usg[i, looper::looper + timeslots - 1] = sgHat
            }
        }
        looper = looper + timeslots
    }

Mata output:

:     for (c=1; c<=draws; c++ ) {
>         for (i=1; i<=rows(statIdLong); i++) {
>             pId = statIdLong[i]
>             if (gameMarker[i] == 1) {
>                 siHat   = asarray(sharesBCS,(pId,c,1))
>                 sgHat   = asarray(sharesBCS,(pId,c,2))
>                 pErr    = asarray(priceErrs,(pId,c))
>                 pB      = asarray(priceBounds,(pId,c))
>             
>                 lnewsp  = lnewsLong[i, ]
>                 otherlp = otherlLong[i, ]
>                 nnewsp  = nnewsLong[i, ]
>             
>                 pop    = popLong[i, ]
>                 logPop = l_ACS_HHLong[i,]
>             
>                 upmt  = upre1[i, looper::looper + timeslots - 1]
>                 ups = upre2[i, looper::looper + timeslots - 1]
>             
>                 meanPHat = otherlp*bpo[1]:*ln(siHat:*pop) :+ lnewsp*bpo[2]:*ln(siHat:*pop) :+ nnewsp*bpo[3]:*ln(siHat:*pop) :+
>                     bpo[4]:*otherlp :+ bpo[5]:*lnewsp :+ bpo[6]:*lo

### Loop to fill in hypothetical viewerships (U - terms, basically)

This is in general a little harder, because of the leads and lags. That's why I am doing it separately. But in principle, this could be combined with the above. Some Notes:

1.    Note that I once again swap `lnews` and `otherl` (that is, put `otherl` where `lnews` "should" be and vice versa) to reflect the counterfactual situation. Don't get confused!

2.    Note that I exponentiate the control terms and subtract one. That's because I saved these as `ln(1 + n)`, but now I will have to modify them in two cases: if a station counterfactually switches to otherl from local news, then I want the count of local news stations to be `ln(1 + n + otherl)`, while I want the count of otherl broadcasts to be `ln(1 + n - otherl)`.  But what if the switching station is broadcasting local news? Then, we will have: `ln(1 + n + otherl - lnews)` and `ln(1 + n + lnews - otherl)`, correct? Let's test it out.

If a station is currently broadcasting local news, and we want counterfactual station counts. Therm 1 will be $(1+n-1)$, while term two will be $(1+n+1)$. Good. If a station is currently broadcasting other, Term 1 will be (1+n+1) and term 2 will be (1+n-1). I like it! 





In [39]:
%%stata -s nh1 --mata
uv  = J(rows(statIdLong), timeslots*draws, 0)
uvg = J(rows(statIdLong), timeslots*draws, 0)
uvb = J(rows(statIdLong), timeslots*draws, 0)

Mata output:




In [40]:
%%stata -s nh1 --mata
looper = 1
    for (c=1; c<=draws; c++ ) {
        for (i=1; i<=rows(statIdLong); i++) {
            pId = statIdLong[i]
            if (gameMarker[i] == 1) {
                siHat   = asarray(sharesBCS, (pId,c,1))
                sgHat   = asarray(sharesBCS, (pId,c,2))
                vErr    = asarray(viewErrs, (pId,c))
                vB      = asarray(viewBounds, (pId,c))   
            
                lnewsp  = lnewsLong[i, ]
                otherlp = otherlLong[i, ]
                nnewsp  = nnewsLong[i, ]
                othercp = othercLong[i, ]
                
                lnewsLp  = lnewsLongLag[i, ]
                otherlLp = otherlLong[i, ]
                nnewsLp  = nnewsLongLag[i, ]
                lnsiLp     = 0, ln(siLong[i,1::5 ])
                totlnewsp = totlnews[i, ]
                totnnewsp = totnnews[i, ]
                lnewsnp   = exp(lnewsnLong[i, ] :- 1)
                nnewsnp   = nnewsnLong[i, ]
                othercnp  = othercnLong[i, ]
                otherlnp  = exp(otherlnLong[i, ] :- 1)
            
                logPop = l_ACS_HHLong[i,]

                uvmt  = uvre1[i, looper::looper + timeslots - 1]
                uvs    = uvre2[i, looper::looper + timeslots - 1]
            
                uvHat = otherlp*bo[5] :+ lnewsp*bo[6] :+ nnewsp*bo[7] :+ 
                        otherlp:*lnewsLp*bo[8] :+ otherlp:*nnewsLp*bo[9] :+
                        nnewsp:*lnewsLp*bo[10] :+ nnewsp:*nnewsLp*bo[11]  :+ 
                        lnsiLp:*bo[12] :+ lnsiLp:*otherlp:*lnewsLp*bo[13] :+
                        lnsiLp:*otherlp:*nnewsLp*bo[14] :+ lnsiLp:*nnewsp:*lnewsLp*bo[15] :+
                        lnsiLp:*nnewsp:*nnewsLp*bo[16] :+ totlnewsp:*bo[17] :+ totnnewsp*bo[18] :+
                        logPop*bo[19] :+ ln(1 :+ lnewsnp :+ otherlp :- lnewsp)*bo[20] :+ 
                        ln(1 :+ lnewsnp :+ lnewsp :- otherlp)*bo[21] :+ nnewsnp*bo[22] :+ othercnp*bo[23] :+ bo[27]
                
                uv[i, looper::looper + timeslots - 1]  = uvHat :+ vErr
                
                muv = otherlp:*bo[1] :+ lnewsp:*bo[2] :+ nnewsp:*bo[3] :+ othercp:*bo[4]
                
                guv =  lnnd(vErr, 0, ln(sdmodv)) :- ln(normal(vB)) :- ln(1 :- muv) :+ 
                        ln(1 :- muv:*siHat:/sgHat :- (1 :- muv):*siHat)
                
                uvg[i, looper::looper + timeslots - 1] = guv
                uvb[i, looper::looper + timeslots - 1] = muv :+ vB
            }
        }
        looper = looper + timeslots
    }

Mata output:

:     for (c=1; c<=draws; c++ ) {
>         for (i=1; i<=rows(statIdLong); i++) {
>             pId = statIdLong[i]
>             if (gameMarker[i] == 1) {
>                 siHat   = asarray(sharesBCS, (pId,c,1))
>                 sgHat   = asarray(sharesBCS, (pId,c,2))
>                 vErr    = asarray(viewErrs, (pId,c))
>                 vB      = asarray(viewBounds, (pId,c))   
>             
>                 lnewsp  = lnewsLong[i, ]
>                 otherlp = otherlLong[i, ]
>                 nnewsp  = nnewsLong[i, ]
>                 othercp = othercLong[i, ]
>                 
>                 lnewsLp  = lnewsLongLag[i, ]
>                 otherlLp = otherlLong[i, ]
>                 nnewsLp  = nnewsLongLag[i, ]
>                 lnsiLp     = 0, ln(siLong[i,1::5 ])
>                 totlnewsp = totlnews[i, ]
>                 totnnewsp = totnnews[i, ]
>                 lnewsnp   = exp(lnewsnLong[i, ] :- 1)
>                 nnewsnp   = nnewsnLong[i, ]
>       

## Game Theoretic stuff...

Drawing errors was hard because of the need for dynamic decisions that we observe to correspond to reality. While computationally, the task is demanding, intuitively it isn't that hard. 

Game Theoretic aspects of the model fall into the same category. We built error terms around the assumption that what we observe is a Nash equilibrium, and now have to check for uniqueness. Otherwise, our model might have a cohesiveness problem, meaning that the likelihood is not unique. So, let's give it a go and see what happens...

**A technical note** for estimation, strictly speaking, the only thing we need to know is how many profiles there are, because we will be using a simple frequency weight. But I am keeping other stuff (like Nash Profits, Shares, Prices, etc.) just because I might want to have a look at them at some point!

In [41]:
%%stata -s nh1 --mata
    NashProfiles=asarray_create("real", 4)
    counter=1

Mata output:

:     NashProfiles=asarray_create("real", 4)

:     counter=1


Here begins the loop, with a schematic to be written, hopefully. What happens is the loop goes through draws, then markets. For each market, it identifies which stations are considered to be in the game. It then fills in stations' information, including error draws. That having been done, it then experiments a bit with perturbation of strategies and then finding best resposes of stations iteratively. 

If the result of this is an equilibrium different from the actual outcome, we record the equilibrium for later use. Anyways:

In [42]:
%%stata -s nh1 --mata
   for (d=1;d<=draws;d++) {
        for (i=1;i<=rows(mLong);i++) {
            gameMarkerp=panelsubmatrix(gameMarker,i,mLong)
            playersp=colsum(gameMarkerp)
            if (playersp>0) {
                statIdLongp=panelsubmatrix(statIdLong,i,mLong)   
                Gamers=select(statIdLongp,gameMarkerp)           
                posofGamers=mm_which(gameMarkerp)                

                UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
                UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
                UvsLongp=panelsubmatrix(UvsLong,i,mLong)
                XBVngLongp=panelsubmatrix(XBVngLong,i,mLong)
                UvmodObsLongp=panelsubmatrix(UvmodObsLong,i,mLong)

                UpmodobsLongp=panelsubmatrix(UpmodObs,i,mLong)
                UpsLongp=panelsubmatrix(UpsLong,i,mLong)
                UpmtLongp=panelsubmatrix(UpmtLong,i,mLong)

                lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
                otherlLongp=panelsubmatrix(otherlLong,i,mLong)
                nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
                othercLongp=panelsubmatrix(othercLong,i,mLong)
                popLongp=panelsubmatrix(popLong,i,mLong)
                l_ACS_HHLongp=panelsubmatrix(l_ACS_HHLong,i,mLong)
                popp = round(exp(max(l_ACS_HHLongp)))    
                lnppsLongp = panelsubmatrix(lnppsLong, i, mLong)

                neq=1

                asarray(NashProfiles,(i,d,1,neq),lnewsLongp)
                asarray(NashProfiles,(i,d,2,neq),otherlLongp)
                asarray(NashProfiles,(i,d,3,neq),nnewsLongp)
                EVdraws=J(rows(UvmodObsLongp),timeslots,0)
                EPdraws=J(rows(UvmodObsLongp),timeslots,0)

                for (z=1; z<=playersp;z++) {
                    EVdraws[posofGamers[z],] = asarray(viewErrs, (Gamers[z],d))
                    EPdraws[posofGamers[z],] = asarray(priceErrs,(Gamers[z],d))
                }

                for (s=1;s<=25;s++) {
                    phlnewsLongp  = lnewsLongp
                    phnnewsLongp  = nnewsLongp
                    photherlLongp = otherlLongp
                    
                    lnewsHat=lnewsLongp
                    nnewsHat=nnewsLongp
                    otherlHat=otherlLongp
                    
                    for (pp=1;pp<=playersp;pp++) {
      
                        targetPlayer=Gamers[pp]
                        lnewsPlayer=asarray(Bcs,(targetPlayer,1))
                        otherlPlayer=asarray(Bcs,(targetPlayer,2))
                        nnewsPlayer=asarray(Bcs,(targetPlayer,3))
                        targetPos=posofGamers[pp]

                        selector = rowsum(lnewsLongp[targetPos,]:==lnewsPlayer) :>= 0
                        lnewsPlayer = select(lnewsPlayer, selector)
                        otherlPlayer = select(otherlPlayer, selector)
                        nnewsPlayer  = select(nnewsPlayer, selector)
  
                        newStrat=round(1+(rows(lnewsPlayer)-1)*runiform(1,1))
  
                        lnewsHat[targetPos,]=lnewsPlayer[newStrat,]
                        nnewsHat[targetPos,]=nnewsPlayer[newStrat,]
                        otherlHat[targetPos,]=otherlPlayer[newStrat,]
                   } 

                   currentvals = J(playersp,1,0)
                   tries = 1
                   maxtries = 20
                   
                   do {
                       lnewsOrig = lnewsHat
                       for (pp=1;pp<=playersp;pp++) {
                           targetPlayer=Gamers[pp]
                           lnewsPlayer=asarray(Bcs,(targetPlayer,1))
                           otherlPlayer=asarray(Bcs,(targetPlayer,2))
                           nnewsPlayer=asarray(Bcs,(targetPlayer,3))
                           targetPos=posofGamers[pp]

                           selector = rowsum(lnewsLongp[targetPos,]:==lnewsPlayer) :>= 0 
                           lnewsPlayer = select(lnewsPlayer, selector)
                           otherlPlayer = select(otherlPlayer, selector)
                           nnewsPlayer  = select(nnewsPlayer, selector)

                           for (g=1;g<=rows(lnewsPlayer);g++) {
                               lnewsOld = lnewsHat
                               nnewsOld = nnewsHat
                               otherlOld = otherlHat

                               lnewsHat[targetPos,]=lnewsPlayer[g,]
                               nnewsHat[targetPos,]=nnewsPlayer[g,]
                               otherlHat[targetPos,]=otherlPlayer[g,]
                    
                               errMarker=(lnewsHat:==phlnewsLongp):*(otherlHat:==photherlLongp)					
                               veToUse = UvmodObsLongp[,counter::counter + timeslots - 1]:*errMarker :+ EVdraws:*(1 :- errMarker)
                               peToUse = UpmodobsLongp[,counter::counter + timeslots - 1]:*errMarker :+ EPdraws:*(1 :- errMarker)
    
                               lnewsLongLagp=J(rows(lnewsLongp),1,0)
                               nnewsLongLagp=J(rows(lnewsLongp),1,0)
                               otherlLongLagp=J(rows(lnewsLongp),1,0)
                               siLagp=J(rows(lnewsLongp),1,0)
                               totlnewsp=J(rows(lnewsLongp),1,0)
                               totnnewsp=J(rows(nnewsLongp),1,0)
    
                               lnewsnLongp=lnewsHat:*ln(1:+colsum(lnewsHat))
                               otherlnLongp=otherlHat:*ln(1:+colsum(otherlHat))
                               nnewsnLongp=nnewsHat:*ln(1:+colsum(nnewsHat))
                               othercnLongp=othercLongp:*ln(1:+colsum(othercLongp))

                               simShares = J(rows(lnewsLongp),timeslots,.)
                               for (t=1;t<=timeslots ;t++) {
                                   if (t!=1) siLagp=ln(siLagp)
                                   XV=lnewsHat[,t],otherlHat[,t],nnewsHat[,t],
                                       lnewsLongLagp:*lnewsHat[,t],
                                       nnewsLongLagp:*lnewsHat[,t],
                                       lnewsLongLagp:*nnewsHat[,t],
                                       nnewsLongLagp:*nnewsHat[,t],
                                       siLagp,
                                       siLagp:*lnewsLongLagp:*lnewsHat[,t],
                                       siLagp:*nnewsLongLagp:*lnewsHat[,t],
                                       siLagp:*lnewsLongLagp:*nnewsHat[,t],
                                       siLagp:*nnewsLongLagp:*nnewsHat[,t],
                                       lnewsHat[,t]:*ln(1:+totlnewsp),nnewsHat[,t]:*ln(1:+totnnewsp),
                                       l_ACS_HHLongp[,t],lnewsnLongp[,t],otherlnLongp[,t],
                                       nnewsnLongp[,t],othercnLongp[,t],J(rows(lnewsHat),1,1)
 
                                   XBV=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                                       UvsLongp[,counter+t-1]:+UvmodObsLongp[,counter+t-1]:+veToUse[,t]

                                   sharesP=esharesStable(XBV,lnewsHat[,t],otherlHat[,t],nnewsHat[,t],
                                       othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
                                   simShares[,t] = sharesP

                                   siLagp=sharesP

                                   totlnewsp=totlnewsp:+J(rows(lnewsHat),1,colsum(lnewsHat[,t]:*sharesP))
                                   totnnewsp=totnnewsp:+J(rows(nnewsHat),1,colsum(nnewsHat[,t]:*sharesP))
                                   lnewsLongLagp=lnewsHat[,t]
                                   nnewsLongLagp=nnewsHat[,t]
                                   otherlLongLagp=otherlHat[,t]
                               }

                               simPrice = (lnewsHat[posofGamers[pp],]*bpo[1]:+otherlHat[posofGamers[pp],]*bpo[2]:+
                                   nnewsHat[posofGamers[pp],]*bpo[3]):*ln(popp:*simShares[posofGamers[pp],]):+
                                   lnewsHat[posofGamers[pp],]:*bpo[4]:+otherlHat[posofGamers[pp],]*bpo[5]:+
                                   bpo[6]:*l_ACS_HHLongp[posofGamers[pp],]:+bpo[10]:+
                                   UpsLongp[posofGamers[pp], counter::counter+timeslots-1]:+
                                   UpmtLongp[posofGamers[pp], counter::counter+timeslots-1] :+
                                   peToUse[posofGamers[pp],]

                               if (rowsum(exp(simPrice)) < currentvals[pp]) {
                                   lnewsHat = lnewsOld
                                   nnewsHat  = nnewsOld
                                   otherlHat = otherlOld
                               }
                               else {
                                   currentvals[pp] = rowsum(exp(simPrice))
                               }
                           }
                       }
                       tries++
                   } while ( (lnewsHat != lnewsOrig) & (tries < maxtries ))
                       if (lnewsHat != phlnewsLongp) {
                           neq++  
                           asarray(NashProfiles,(i,d,1,neq),lnewsHat)
                           asarray(NashProfiles,(i,d,2,neq),otherlHat)
                           asarray(NashProfiles,(i,d,3,neq),nnewsHat)
                       }
                   }
                   lnewsLongp  = phlnewsLongp
                   nnewsLongp  = nnewsLongp
                   otherlLongp = photherlLongp
                }
            }
        }
        counter = counter + timeslots
    }


Mata output:




Finally, save everything and continue to the next workbook, which collects all of our draws and things into a format that can be used by the Ackerberg importance sampler. 

In [43]:
%%stata -s nh1 --mata
    mata matsave NashStuff NashProfiles, replace

Mata output:

:     mata matsave NashStuff NashProfiles, replace
(saving NashProfiles)
file NashStuff.mmat saved


And dont forget to save all the game-theoretic stuff for easy estimation!

In [44]:
%%stata -s nh1 --mata
    mata matsave usANDgs upre1 upre2 uvre1 uvre2 upre1g upre2g uvre1g uvre2g up upg upb uv uvg uvb usi usg, replace

Mata output:

:     mata matsave usANDgs upre1 upre2 uvre1 uvre2 upre1g upre2g uvre1g uvre2g up upg upb uv uvg uvb usi usg, replace
(saving up[16622,120], upb[16622,120], upg[16622,120], upre1[16622,120], upre1g[16622,120], upre2[16622,120], upre2g[16622,120],
 usg[16622,120], usi[16622,120], uv[16622,120], uvb[16622,120], uvg[16622,120], uvre1[16622,120], uvre1g[16622,120], uvre2[16622,120],
 uvre2g[16622,120])
file usANDgs.mmat saved


## Odd thing...
I have had more success running this whole file in Stata and not in the ipython notebook. For some reason, it doesn't seem like the serialized object `NashProfiles` saves correctly